In [1]:
import gradio as gr
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch

In [2]:
# Load model and processor
model_name = "dima806/deepfake_vs_real_image_detection"
model = AutoModelForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
id2label = {
    "0": "fake",
    "1": "real"
}

In [4]:
def classify_image(image):
    image = Image.fromarray(image).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()

    prediction = {
        id2label[str(i)]: round(probs[i], 3) for i in range(len(probs))
    }

    return prediction

In [5]:
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Label(num_top_classes=2, label="Deepfake Detection"),
    title="UDFD",
    description="Upload an image to detect whether it is AI-generated (Fake) or a real photograph (Real)."
)

In [6]:
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
#todo: Explainability con gradcam, shap o lime per evidenziare porzioni di immagini che possano aver influenzato l'output
#todo: La web app finale può contenere l'interfaccia (UI) del modello
#todo: Modificare il codice del modello per integrare un modulo di sicurezza per identificare eventuali patter posinous
#todo: Etichettare il dataset nostro per eventualmente bilanciarlo con FairSmote
#todo: MLOps aggiungere pipeline di addestramento automatico, per esempio dando la possibilità agli utenti di valutare l'output (Va gestito il caricamento delle immagini che vanno salvate sul server), dopo un tot numero di NO si riaddestra (Questa cosa può collegarsi con il modulo di sicurezza). Se l'utente non è collaborativo c'è il checkpointing.